# 🔴 Avancé | ⏱ 45 min | 🔑 Concepts : with, __enter__, __exit__, @contextmanager

# Context Managers en Python

## 🎯 Objectifs

- Comprendre le protocole de context manager
- Implémenter `__enter__` et `__exit__` dans une classe
- Utiliser `@contextmanager` de contextlib
- Découvrir les utilitaires de contextlib
- Maîtriser les cas d'usage avancés
- Créer des context managers personnalisés

## 📋 Prérequis

- Python 3.8+
- Connaissances en POO (classes, méthodes spéciales)
- Familiarité avec `with` statement
- Compréhension des exceptions

## 1. Rappel : Pourquoi les context managers ?

Les context managers garantissent qu'une ressource est **correctement acquise et libérée**, même en cas d'erreur.

**Problèmes sans context manager** :
- Oubli de libérer une ressource
- Ressource non libérée en cas d'exception
- Code verbeux avec try/finally

**Ressources typiques** :
- Fichiers
- Connexions réseau/base de données
- Locks/sémaphores
- Transactions
- Timers

In [ ]:
import tempfile
from pathlib import Path

temp_dir = Path(tempfile.mkdtemp(prefix='cm_demo_'))
fichier = temp_dir / 'test.txt'

# ❌ SANS context manager (risqué)
print("❌ Sans context manager :\n")
f = open(fichier, 'w')
f.write('Données importantes')
# Si erreur ici, le fichier reste ouvert !
f.close()
print("  Problème : si erreur avant close(), fuite de ressource\n")

# ❌ AVEC try/finally (verbeux)
print("❌ Avec try/finally (verbeux) :\n")
f = None
try:
    f = open(fichier, 'w')
    f.write('Données importantes')
finally:
    if f:
        f.close()
print("  Fonctionne mais très verbeux\n")

# ✅ AVEC context manager (élégant)
print("✅ Avec context manager (élégant) :\n")
with open(fichier, 'w') as f:
    f.write('Données importantes')
    # f.close() appelé automatiquement
print("  ✅ Concis et sûr !")
print(f"  Fichier fermé ? {f.closed}")

## 2. Le protocole context manager

Un context manager est un objet qui implémente deux méthodes spéciales :

- `__enter__(self)` : Appelée au début du bloc `with`, retourne la ressource
- `__exit__(self, exc_type, exc_val, exc_tb)` : Appelée à la fin du bloc (même en cas d'erreur)

### Signature de `__exit__`

```python
def __exit__(self, exc_type, exc_val, exc_tb):
    # exc_type : type de l'exception (ou None)
    # exc_val : instance de l'exception (ou None)
    # exc_tb : traceback (ou None)
    # Return True pour supprimer l'exception
```

## 3. Créer un context manager avec une classe

In [ ]:
# Exemple simple : gestionnaire de fichier personnalisé
class ManagedFile:
    """Context manager pour gérer l'ouverture/fermeture de fichiers."""
    
    def __init__(self, filename, mode='r'):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        """Appelé au début du bloc with."""
        print(f"  → Ouverture de {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file  # Retourné à la variable après 'as'
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Appelé à la fin du bloc with."""
        print(f"  → Fermeture de {self.filename}")
        
        if self.file:
            self.file.close()
        
        # Si une exception s'est produite
        if exc_type is not None:
            print(f"  ⚠️ Exception capturée : {exc_type.__name__}: {exc_val}")
        
        # Return False : l'exception est re-levée
        # Return True : l'exception est supprimée
        return False  # Re-lever l'exception

# Test
fichier_test = temp_dir / 'managed.txt'
fichier_test.write_text('Contenu initial')

print("Test du context manager personnalisé :\n")
with ManagedFile(fichier_test, 'r') as f:
    contenu = f.read()
    print(f"  Contenu : {contenu}")

print("\n✅ Fichier fermé automatiquement")

In [ ]:
# Test avec une exception
print("\nTest avec exception :\n")
try:
    with ManagedFile(fichier_test, 'r') as f:
        contenu = f.read()
        print(f"  Contenu : {contenu}")
        raise ValueError("Erreur simulée")
        print("  Ceci ne sera jamais exécuté")
except ValueError as e:
    print(f"\n✅ Exception propagée : {e}")
    print("✅ Mais le fichier a bien été fermé !")

### 3.1 Exemple : Timer context manager

In [ ]:
import time

class Timer:
    """Context manager pour mesurer le temps d'exécution."""
    
    def __init__(self, name="Opération"):
        self.name = name
        self.start_time = None
        self.elapsed = None
    
    def __enter__(self):
        print(f"⏱️  Démarrage : {self.name}")
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.elapsed = time.time() - self.start_time
        print(f"⏱️  Fin : {self.name}")
        print(f"   Durée : {self.elapsed:.4f} secondes")
        return False

# Test
print("Test du Timer :\n")

with Timer("Calcul lourd"):
    # Simuler un calcul
    total = sum(i ** 2 for i in range(1000000))
    print(f"   Résultat : {total}")

print("\n" + "="*50 + "\n")

with Timer("Sleep"):
    time.sleep(0.5)
    print("   Sleep terminé")

### 3.2 Exemple : Database connection (mock)

In [ ]:
class DatabaseConnection:
    """Mock d'une connexion base de données avec transaction."""
    
    def __init__(self, db_name):
        self.db_name = db_name
        self.connected = False
        self.transaction_active = False
    
    def __enter__(self):
        print(f"📊 Connexion à la base '{self.db_name}'")
        self.connected = True
        self.transaction_active = True
        print(f"📊 Transaction démarrée")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type is None:
            # Pas d'erreur : commit
            print(f"✅ COMMIT de la transaction")
        else:
            # Erreur : rollback
            print(f"❌ ROLLBACK de la transaction (erreur: {exc_type.__name__})")
        
        self.transaction_active = False
        self.connected = False
        print(f"📊 Déconnexion de '{self.db_name}'")
        
        return False  # Propager l'exception
    
    def execute(self, query):
        """Simuler l'exécution d'une requête."""
        print(f"   SQL : {query}")

# Test sans erreur
print("Test 1 : Transaction réussie\n")
with DatabaseConnection('production') as db:
    db.execute("INSERT INTO users (name) VALUES ('Alice')")
    db.execute("INSERT INTO users (name) VALUES ('Bob')")

print("\n" + "="*60 + "\n")

# Test avec erreur
print("Test 2 : Transaction avec erreur\n")
try:
    with DatabaseConnection('production') as db:
        db.execute("INSERT INTO users (name) VALUES ('Charlie')")
        raise ValueError("Contrainte violée")
        db.execute("Ceci ne sera jamais exécuté")
except ValueError:
    print("\n⚠️ Exception gérée par le code appelant")

## 4. @contextmanager : Approche avec générateur

`@contextmanager` de `contextlib` permet de créer un context manager en utilisant un **générateur** au lieu d'une classe.

**Avantages** :
- Plus concis
- Moins de boilerplate
- Code plus lisible pour des cas simples

**Structure** :
```python
from contextlib import contextmanager

@contextmanager
def my_context():
    # Code __enter__
    yield ressource  # Retourné à 'as'
    # Code __exit__
```

In [ ]:
from contextlib import contextmanager
import time

# Version générateur du Timer
@contextmanager
def timer(name="Opération"):
    """Context manager timer avec générateur."""
    print(f"⏱️  Démarrage : {name}")
    start = time.time()
    
    try:
        yield  # Point de séparation __enter__/__exit__
    finally:
        # Code __exit__ (toujours exécuté)
        elapsed = time.time() - start
        print(f"⏱️  Fin : {name}")
        print(f"   Durée : {elapsed:.4f} secondes")

# Test
print("Test du timer avec @contextmanager :\n")

with timer("Calcul"):
    result = sum(range(1000000))
    print(f"   Résultat : {result}")

In [ ]:
from contextlib import contextmanager

# Context manager avec valeur retournée
@contextmanager
def timer_with_result(name="Opération"):
    """Timer qui retourne un objet pour stocker la durée."""
    class TimerResult:
        def __init__(self):
            self.elapsed = None
    
    result = TimerResult()
    print(f"⏱️  Démarrage : {name}")
    start = time.time()
    
    try:
        yield result  # Objet retourné à 'as'
    finally:
        result.elapsed = time.time() - start
        print(f"⏱️  Fin : {name} ({result.elapsed:.4f}s)")

# Test
print("\nTimer avec résultat :\n")

with timer_with_result("Opération 1") as t1:
    time.sleep(0.1)

with timer_with_result("Opération 2") as t2:
    time.sleep(0.2)

print(f"\nComparaison :")
print(f"  Op1 : {t1.elapsed:.4f}s")
print(f"  Op2 : {t2.elapsed:.4f}s")
print(f"  Différence : {abs(t2.elapsed - t1.elapsed):.4f}s")

### 4.1 Exemple : Changer temporairement un état

In [ ]:
from contextlib import contextmanager
import os

@contextmanager
def temporary_directory(path):
    """Change temporairement le répertoire de travail."""
    original_dir = os.getcwd()
    print(f"📁 Changement : {original_dir} → {path}")
    
    try:
        os.chdir(path)
        yield path
    finally:
        os.chdir(original_dir)
        print(f"📁 Retour : {path} → {original_dir}")

# Test
print("Test de changement temporaire de répertoire :\n")
print(f"Répertoire actuel : {os.getcwd()}\n")

with temporary_directory(temp_dir):
    print(f"Dans le bloc : {os.getcwd()}")
    # Faire des opérations dans ce dossier

print(f"\nAprès le bloc : {os.getcwd()}")

## 5. contextlib : Utilitaires avancés

Le module `contextlib` fournit des utilitaires pour travailler avec les context managers.

### 5.1 suppress : Supprimer des exceptions

In [ ]:
from contextlib import suppress
import os

# Sans suppress
print("Sans suppress :\n")
try:
    os.remove('fichier_inexistant.txt')
    print("  Fichier supprimé")
except FileNotFoundError:
    print("  ⚠️ Fichier inexistant (erreur capturée)")

# Avec suppress (plus concis)
print("\nAvec suppress :\n")
with suppress(FileNotFoundError):
    os.remove('fichier_inexistant.txt')
    print("  Fichier supprimé")

print("  ✅ Pas d'erreur levée")

# Utile pour cleanup
print("\nExemple : cleanup de plusieurs fichiers\n")
fichiers_a_supprimer = ['temp1.txt', 'temp2.txt', 'temp3.txt']

for fichier in fichiers_a_supprimer:
    with suppress(FileNotFoundError):
        os.remove(fichier)
        print(f"  ✅ {fichier} supprimé")

print("\n✅ Cleanup terminé sans erreur")

### 5.2 redirect_stdout / redirect_stderr

In [ ]:
from contextlib import redirect_stdout, redirect_stderr
import io
import sys

# Capturer stdout
print("Test de redirect_stdout :\n")

output = io.StringIO()
with redirect_stdout(output):
    print("Ceci est capturé")
    print("Ligne 2")
    print("Ligne 3")

captured = output.getvalue()
print(f"Sortie capturée :\n{captured}")

# Rediriger vers un fichier
print("\nRedirection vers un fichier :\n")
fichier_log = temp_dir / 'output.log'

with open(fichier_log, 'w') as f:
    with redirect_stdout(f):
        print("Log ligne 1")
        print("Log ligne 2")
        print(f"Timestamp : {time.time()}")

print(f"✅ Logs écrits dans {fichier_log.name}")
print(f"Contenu :\n{fichier_log.read_text()}")

### 5.3 closing : Assurer la fermeture

In [ ]:
from contextlib import closing
from urllib.request import urlopen

# Pour des objets qui ont close() mais pas de support with
class MyResource:
    def __init__(self, name):
        self.name = name
        print(f"  Ressource '{name}' créée")
    
    def do_something(self):
        print(f"  Utilisation de '{self.name}'")
    
    def close(self):
        print(f"  Ressource '{self.name}' fermée")

# Sans closing : risque d'oubli
print("Sans closing :\n")
resource = MyResource('res1')
resource.do_something()
resource.close()  # Facile à oublier !

# Avec closing : sûr
print("\nAvec closing :\n")
with closing(MyResource('res2')) as resource:
    resource.do_something()
    # close() appelé automatiquement

print("\n✅ Fermeture automatique garantie")

### 5.4 ExitStack : Empiler des context managers

In [ ]:
from contextlib import ExitStack

# Problème : nombre variable de fichiers à ouvrir
print("Test d'ExitStack :\n")

# Créer des fichiers de test
fichiers = []
for i in range(5):
    f = temp_dir / f'file{i}.txt'
    f.write_text(f'Contenu du fichier {i}')
    fichiers.append(f)

# Sans ExitStack : nested with très verbeux
# with open(fichiers[0]) as f1:
#     with open(fichiers[1]) as f2:
#         with open(fichiers[2]) as f3:
#             ...

# Avec ExitStack : élégant
print("Ouverture de plusieurs fichiers avec ExitStack :\n")

with ExitStack() as stack:
    # Ouvrir tous les fichiers
    files = [stack.enter_context(open(f, 'r')) for f in fichiers]
    
    print(f"  ✅ {len(files)} fichiers ouverts")
    
    # Lire le contenu
    for i, f in enumerate(files):
        content = f.read()
        print(f"    Fichier {i}: {content}")
    
    # Tous seront fermés automatiquement

print("\n✅ Tous les fichiers ont été fermés")

In [ ]:
from contextlib import ExitStack

# Exemple : copier plusieurs fichiers avec gestion d'erreur
print("\nExemple : traitement conditionnel avec ExitStack\n")

def process_files_conditionally(files_to_process):
    """
    Ouvre et traite des fichiers conditionnellement.
    """
    with ExitStack() as stack:
        opened_files = []
        
        for filepath in files_to_process:
            try:
                f = stack.enter_context(open(filepath, 'r'))
                opened_files.append((filepath, f))
                print(f"  ✅ Ouvert : {filepath.name}")
            except FileNotFoundError:
                print(f"  ⚠️ Ignoré : {filepath.name} (inexistant)")
        
        # Traiter les fichiers ouverts
        print(f"\n  Traitement de {len(opened_files)} fichiers :")
        for path, f in opened_files:
            content = f.read()
            print(f"    {path.name}: {len(content)} caractères")
        
        # Tous fermés automatiquement à la sortie

# Test
files_to_test = [
    fichiers[0],
    temp_dir / 'inexistant.txt',
    fichiers[1],
    fichiers[2],
]

process_files_conditionally(files_to_test)
print("\n✅ Tous les fichiers ouverts ont été fermés")

## 6. Cas d'usage avancés

### 6.1 Lock threading

In [ ]:
import threading
import time

# Lock pour synchronisation
lock = threading.Lock()
counter = 0

def increment_with_lock():
    global counter
    # ❌ Sans with (risqué)
    # lock.acquire()
    # counter += 1
    # lock.release()  # Peut être oublié !
    
    # ✅ Avec with (sûr)
    with lock:
        current = counter
        time.sleep(0.001)  # Simuler un traitement
        counter = current + 1

print("Test de Lock avec context manager :\n")

# Créer plusieurs threads
threads = []
for i in range(10):
    t = threading.Thread(target=increment_with_lock)
    threads.append(t)
    t.start()

# Attendre la fin
for t in threads:
    t.join()

print(f"  Valeur finale du compteur : {counter}")
print("  ✅ Synchronisation réussie (attendu: 10)")

### 6.2 Nested context managers

In [ ]:
# Multiples context managers sur une ligne
print("Context managers multiples :\n")

source = temp_dir / 'source.txt'
dest = temp_dir / 'dest.txt'
source.write_text('Données à copier')

# Méthode 1 : Sur une ligne (Python 3.1+)
with open(source, 'r') as f_in, open(dest, 'w') as f_out:
    content = f_in.read()
    f_out.write(content.upper())

print(f"  ✅ Copié et converti en majuscules")
print(f"  Résultat : {dest.read_text()}")

# Méthode 2 : Nested (plus lisible si beaucoup de contextes)
print("\nAvec timer imbriqué :\n")

with timer("Copie de fichier"):
    with open(source, 'r') as f_in:
        with open(dest, 'w') as f_out:
            content = f_in.read()
            f_out.write(content)

## 7. Async context managers (preview)

Python 3.5+ supporte aussi les **async context managers** avec `async with`.

Méthodes spéciales :
- `__aenter__` : async version de `__enter__`
- `__aexit__` : async version de `__exit__`

In [ ]:
# Exemple (code pour référence, ne s'exécute pas dans ce notebook)
code_async = '''
import asyncio

class AsyncDatabaseConnection:
    """Async context manager pour connexion DB."""
    
    async def __aenter__(self):
        print("Connexion async...")
        await asyncio.sleep(0.1)  # Simuler I/O
        return self
    
    async def __aexit__(self, exc_type, exc_val, exc_tb):
        print("Déconnexion async...")
        await asyncio.sleep(0.1)
        return False
    
    async def query(self, sql):
        print(f"Exécution : {sql}")
        await asyncio.sleep(0.05)
        return ["result1", "result2"]

async def main():
    async with AsyncDatabaseConnection() as db:
        results = await db.query("SELECT * FROM users")
        print(f"Résultats : {results}")

# Exécution
asyncio.run(main())
'''

print("Exemple d'async context manager :\n")
print(code_async)
print("\n→ Utile pour I/O asynchrones (réseau, fichiers, DB)")

## 🚨 Pièges courants

### 1. Oublier __exit__ même en cas d'exception

In [ ]:
# ❌ MAUVAIS : __exit__ qui peut échouer
class BadContextManager:
    def __enter__(self):
        print("  __enter__")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # Problème : si ceci échoue, l'exception originale est masquée !
        raise RuntimeError("Erreur dans __exit__")

print("❌ Context manager avec __exit__ qui échoue :\n")
try:
    with BadContextManager():
        raise ValueError("Erreur originale")
except RuntimeError as e:
    print(f"  Erreur capturée : {e}")
    print("  ❌ L'erreur originale (ValueError) a été masquée !\n")

# ✅ BON : __exit__ robuste
class GoodContextManager:
    def __enter__(self):
        print("  __enter__")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            # Cleanup qui peut échouer
            print("  Cleanup dans __exit__")
            # ... cleanup ...
        except Exception as e:
            print(f"  ⚠️ Erreur de cleanup (non fatale) : {e}")
        
        # Toujours retourner False pour propager l'exception originale
        return False

print("✅ Context manager robuste :\n")
try:
    with GoodContextManager():
        raise ValueError("Erreur originale")
except ValueError as e:
    print(f"  ✅ Erreur originale capturée : {e}")

### 2. Yield unique avec @contextmanager

In [ ]:
from contextlib import contextmanager

# ❌ ERREUR : plusieurs yield
@contextmanager
def bad_context():
    print("Setup")
    yield "value1"
    # yield "value2"  # ERREUR : un seul yield autorisé !
    print("Cleanup")

print("Avec @contextmanager : un seul yield !\n")
with bad_context() as value:
    print(f"  Valeur : {value}")

print("\n→ @contextmanager ne permet qu'UN SEUL yield")

## 💪 Mini-exercices

### Exercice 1 : Timer context manager avec statistiques

Créer un context manager qui mesure le temps ET collecte des statistiques.

In [ ]:
import time
from typing import List

class TimerStats:
    """
    Context manager qui mesure le temps et collecte des stats.
    """
    
    def __init__(self):
        self.measurements: List[float] = []
    
    def timer(self, name: str):
        # TODO: retourner un context manager pour mesurer
        pass
    
    def report(self):
        # TODO: afficher les statistiques
        pass

# Test
# stats = TimerStats()
# 
# with stats.timer("Opération 1"):
#     time.sleep(0.1)
# 
# with stats.timer("Opération 2"):
#     time.sleep(0.2)
# 
# stats.report()

### Solution Exercice 1

In [ ]:
import time
from typing import List, Dict
from contextlib import contextmanager

class TimerStats:
    """
    Context manager qui mesure le temps et collecte des stats.
    """
    
    def __init__(self):
        self.measurements: Dict[str, List[float]] = {}
    
    @contextmanager
    def timer(self, name: str):
        """Mesure le temps d'exécution d'un bloc."""
        start = time.time()
        
        try:
            yield
        finally:
            elapsed = time.time() - start
            
            if name not in self.measurements:
                self.measurements[name] = []
            
            self.measurements[name].append(elapsed)
            print(f"⏱️  {name}: {elapsed:.4f}s")
    
    def report(self):
        """Affiche les statistiques."""
        if not self.measurements:
            print("Aucune mesure enregistrée")
            return
        
        print("\n" + "="*60)
        print("RAPPORT DE PERFORMANCE")
        print("="*60)
        
        for name, times in self.measurements.items():
            count = len(times)
            total = sum(times)
            avg = total / count
            min_time = min(times)
            max_time = max(times)
            
            print(f"\n{name}:")
            print(f"  Exécutions : {count}")
            print(f"  Total      : {total:.4f}s")
            print(f"  Moyenne    : {avg:.4f}s")
            print(f"  Min        : {min_time:.4f}s")
            print(f"  Max        : {max_time:.4f}s")
        
        print("\n" + "="*60)

# Test
print("Test du TimerStats :\n")

stats = TimerStats()

# Plusieurs mesures
for i in range(3):
    with stats.timer("Calcul rapide"):
        sum(range(100000))

for i in range(2):
    with stats.timer("Sleep 0.1s"):
        time.sleep(0.1)

with stats.timer("Calcul lourd"):
    sum(range(1000000))

# Rapport
stats.report()

### Exercice 2 : Database connection mock avec transaction

Créer un context manager qui simule une connexion DB avec gestion de transaction.

In [ ]:
class Database:
    """
    Mock de base de données avec transactions.
    """
    
    def __init__(self):
        self.data = []  # Stockage en mémoire
        self.in_transaction = False
        self.transaction_buffer = []
    
    def __enter__(self):
        # TODO: démarrer une transaction
        pass
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        # TODO: commit ou rollback
        pass
    
    def insert(self, value):
        # TODO: ajouter à la transaction
        pass

# Test
# db = Database()
# 
# # Transaction réussie
# with db:
#     db.insert("Alice")
#     db.insert("Bob")
# 
# # Transaction échouée
# try:
#     with db:
#         db.insert("Charlie")
#         raise ValueError("Erreur")
# except ValueError:
#     pass
# 
# print(db.data)  # ['Alice', 'Bob'] (Charlie n'a pas été ajouté)

### Solution Exercice 2

In [ ]:
class Database:
    """
    Mock de base de données avec transactions.
    """
    
    def __init__(self, name="MyDB"):
        self.name = name
        self.data = []  # Stockage en mémoire
        self.in_transaction = False
        self.transaction_buffer = []
    
    def __enter__(self):
        """Démarre une transaction."""
        print(f"\n🔵 BEGIN TRANSACTION ({self.name})")
        self.in_transaction = True
        self.transaction_buffer = []
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Commit ou rollback selon le résultat."""
        if exc_type is None:
            # Pas d'erreur : COMMIT
            self.data.extend(self.transaction_buffer)
            print(f"✅ COMMIT ({len(self.transaction_buffer)} insertions)")
        else:
            # Erreur : ROLLBACK
            print(f"❌ ROLLBACK (erreur: {exc_type.__name__}: {exc_val})")
        
        self.in_transaction = False
        self.transaction_buffer = []
        return False  # Propager l'exception
    
    def insert(self, value):
        """Insère une valeur (dans la transaction si active)."""
        if self.in_transaction:
            self.transaction_buffer.append(value)
            print(f"  INSERT (transaction): {value}")
        else:
            self.data.append(value)
            print(f"  INSERT (direct): {value}")
    
    def select_all(self):
        """Retourne toutes les données."""
        return self.data.copy()
    
    def __repr__(self):
        return f"Database({self.name}): {len(self.data)} éléments"

# Test complet
print("="*60)
print("TEST DE LA BASE DE DONNÉES AVEC TRANSACTIONS")
print("="*60)

db = Database("TestDB")

print("\n1. Transaction réussie :")
with db:
    db.insert("Alice")
    db.insert("Bob")
    db.insert("Charlie")

print(f"\nDonnées actuelles : {db.select_all()}")

print("\n2. Transaction échouée (rollback) :")
try:
    with db:
        db.insert("David")
        db.insert("Eve")
        raise ValueError("Contrainte violée")
        db.insert("Frank")  # Ne sera jamais exécuté
except ValueError:
    print("\n⚠️ Exception gérée par le code appelant")

print(f"\nDonnées actuelles : {db.select_all()}")
print("→ David et Eve n'ont PAS été ajoutés (rollback)")

print("\n3. Insertion directe (sans transaction) :")
db.insert("Grace")

print(f"\nDonnées finales : {db.select_all()}")
print(f"\n{db}")

print("\n" + "="*60)
print("✅ Test terminé")
print("="*60)

### Exercice 3 : File handler personnalisé

Créer un context manager qui encapsule l'ouverture de fichier avec logging.

In [ ]:
from contextlib import contextmanager
from pathlib import Path
from typing import Optional

@contextmanager
def logged_file(filepath: str, mode: str = 'r', encoding: str = 'utf-8'):
    """
    Context manager pour fichier avec logging.
    
    Log:
    - Ouverture/fermeture
    - Temps d'ouverture
    - Erreurs éventuelles
    """
    # TODO: implémenter
    pass

# Test
# with logged_file('test.txt', 'w') as f:
#     f.write('Hello')

### Solution Exercice 3

In [ ]:
from contextlib import contextmanager
from pathlib import Path
import time
from datetime import datetime

@contextmanager
def logged_file(filepath: str, mode: str = 'r', encoding: str = 'utf-8'):
    """
    Context manager pour fichier avec logging détaillé.
    """
    path = Path(filepath)
    start_time = time.time()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    print(f"\n[{timestamp}] 📂 Ouverture : {path.name}")
    print(f"  Mode : {mode}")
    print(f"  Encoding : {encoding}")
    
    if path.exists():
        size = path.stat().st_size
        print(f"  Taille actuelle : {size} octets")
    else:
        print(f"  Fichier inexistant (sera créé)")
    
    file = None
    try:
        file = open(filepath, mode, encoding=encoding)
        yield file
    except Exception as e:
        print(f"  ❌ Erreur : {type(e).__name__}: {e}")
        raise
    finally:
        if file:
            file.close()
        
        elapsed = time.time() - start_time
        timestamp_end = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        print(f"[{timestamp_end}] 🔒 Fermeture : {path.name}")
        print(f"  Durée d'ouverture : {elapsed:.4f}s")
        
        if path.exists():
            final_size = path.stat().st_size
            print(f"  Taille finale : {final_size} octets")

# Test complet
print("="*60)
print("TEST DU FILE HANDLER AVEC LOGGING")
print("="*60)

test_file = temp_dir / 'logged_file.txt'

print("\n1. Écriture :")
with logged_file(test_file, 'w') as f:
    f.write("Ligne 1\n")
    f.write("Ligne 2\n")
    time.sleep(0.1)  # Simuler un traitement
    f.write("Ligne 3\n")

print("\n2. Lecture :")
with logged_file(test_file, 'r') as f:
    content = f.read()
    print(f"\n  Contenu lu :\n{content}")

print("\n3. Ajout :")
with logged_file(test_file, 'a') as f:
    f.write("Ligne ajoutée\n")

print("\n4. Lecture finale :")
with logged_file(test_file, 'r') as f:
    lines = f.readlines()
    print(f"\n  Nombre de lignes : {len(lines)}")
    for i, line in enumerate(lines, 1):
        print(f"    {i}. {line.rstrip()}")

print("\n" + "="*60)
print("✅ Test terminé")
print("="*60)

## 📚 Ressources complémentaires

- [PEP 343 - The "with" Statement](https://www.python.org/dev/peps/pep-0343/)
- [Documentation contextlib](https://docs.python.org/3/library/contextlib.html)
- [Real Python - Context Managers](https://realpython.com/python-with-statement/)
- [PEP 492 - Coroutines with async and await](https://www.python.org/dev/peps/pep-0492/)
- [Effective Python - Item 66: Use contextlib](https://effectivepython.com/)

## 💡 Conseils

1. **Toujours utiliser context managers** pour les ressources (fichiers, connexions, locks)
2. **@contextmanager pour simplicité** : préférer les générateurs aux classes pour les cas simples
3. **__exit__ robuste** : gérer les erreurs dans __exit__ sans masquer l'exception originale
4. **Return False par défaut** : dans __exit__, toujours retourner False sauf si vous voulez vraiment supprimer l'exception
5. **ExitStack pour dynamique** : quand le nombre de ressources n'est pas connu à l'avance
6. **Tester les cas d'erreur** : vérifier que les ressources sont bien libérées même en cas d'exception
7. **Documentation claire** : documenter ce qui est acquis/libéré et dans quel ordre